### Download and prepare KDD1999 10% dataset

In [1]:
import os
import pandas as pd
import numpy as np
from urllib.request import urlretrieve
import zipfile
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import silhouette_score

def load_kddcup_data(filepath):
    columns = [
        'duration', 'protocol_type', 'service', 'flag', 'src_bytes', 'dst_bytes',
        'land', 'wrong_fragment', 'urgent', 'hot', 'num_failed_logins', 'logged_in',
        'num_compromised', 'root_shell', 'su_attempted', 'num_root', 'num_file_creations',
        'num_shells', 'num_access_files', 'num_outbound_cmds', 'is_host_login', 'is_guest_login',
        'count', 'srv_count', 'serror_rate', 'srv_serror_rate', 'rerror_rate', 'srv_rerror_rate',
        'same_srv_rate', 'diff_srv_rate', 'srv_diff_host_rate', 'dst_host_count', 'dst_host_srv_count',
        'dst_host_same_srv_rate', 'dst_host_diff_srv_rate', 'dst_host_same_src_port_rate',
        'dst_host_srv_diff_host_rate', 'dst_host_serror_rate', 'dst_host_srv_serror_rate',
        'dst_host_rerror_rate', 'dst_host_srv_rerror_rate', 'label'
    ]
    
    data = pd.read_csv(filepath, header=None, names=columns)
    return data

def preprocess_kddcup_data(data):
    # Separate features and labels
    X = data.drop(columns=['label'])
    
    # Define categorical and numerical columns
    categorical_features = ['protocol_type', 'service', 'flag']
    numerical_features = [col for col in X.columns if col not in categorical_features]
    
    # Preprocessing pipeline
    preprocessor = ColumnTransformer(
        transformers=[
            # Convert all numerical features to standard normal destribution
            ('num', StandardScaler(), numerical_features),
            # One hot encode categorical features
            # ('cat', OneHotEncoder(), categorical_features)
        ]
    )
    
    return preprocessor.fit_transform(X)

def download_kddcup99(url, destination_file_name, destination_folder="/home/jovyan/work/data"):
    if not os.path.exists(destination_folder):
        os.makedirs(destination_folder)
        
    compressed_file = os.path.join(destination_folder, destination_file_name + ".gz")
    extracted_file = os.path.join(destination_folder, destination_file_name)
    
    # Download the dataset if not already downloaded
    if not os.path.exists(compressed_file):
        print("Downloading dataset...")
        urlretrieve(url, compressed_file)
        print("Download complete.")
    
    # Extract the dataset if not already extracted
    if not os.path.exists(extracted_file):
        print("Extracting dataset...")
        os.system(f"gunzip {compressed_file}")
        print("Extraction complete.")
    
    return extracted_file

In [5]:
url = "http://kdd.ics.uci.edu/databases/kddcup99/kddcup.data_10_percent.gz"
data_file_path = download_kddcup99(url, "kdd_10_percent")


In [6]:
k = 5  # Number of clusters
max_iterations = 20
num_partitions = 10

In [7]:
data = load_kddcup_data(data_file_path)
processed_data = preprocess_kddcup_data(data)

### Simple KMeans

In [8]:
import time
import numpy as np
import matplotlib.pyplot as plt

def initialize_centroids(X, k, seed=314):
    """
    Randomly initialize centroids from the dataset with a fixed seed for reproducibility.
    """
    np.random.seed(seed)
    random_indices = np.random.choice(X.shape[0], k, replace=False)
    centroids = X[random_indices]
    return centroids

def compute_distances(X, centroids):
    """
    Compute the distance between each data point and each centroid.
    """
    distances = np.zeros((X.shape[0], len(centroids)))
    for i, centroid in enumerate(centroids):
        distances[:, i] = np.linalg.norm(X - centroid, axis=1)
    return distances

def assign_clusters(distances):
    """
    Assign each data point to the closest centroid.
    """
    return np.argmin(distances, axis=1)

def update_centroids(X, labels, k):
    """
    Update centroids as the mean of all points assigned to each cluster.
    """
    centroids = np.zeros((k, X.shape[1]))
    for i in range(k):
        points = X[labels == i]
        if points.shape[0] > 0:
            centroids[i] = points.mean(axis=0)
    return centroids

def simple_kmeans(X, k, max_iters=20, tolerance=1e-4):
    """
    Perform the K-Means clustering algorithm.
    """
    start_time = time.time()
    centroids = initialize_centroids(X, k)
    for i in range(max_iters):
        iteration_time = time.time()
        old_centroids = centroids
        distances = compute_distances(X, centroids)
        labels = assign_clusters(distances)
        centroids = update_centroids(X, labels, k)

        print(f"Iteration: {i}\ttime taken: {time.time() - iteration_time}")
        
        # Check for convergence
        if np.all(np.abs(centroids - old_centroids) < tolerance):
            print(f"K-Means converged after {i+1} iterations.")
            break
    print(f"Total time: {time.time() - start_time}")
    return centroids, labels

In [ ]:
# centroids, labels = simple_kmeans(processed_data, k)

In [7]:
from pyspark.sql import SparkSession
from pyspark.ml.evaluation import ClusteringEvaluator
from pyspark.ml.linalg import Vectors
from pyspark.sql import Row

# Create Spark session
spark = SparkSession.builder \
    .appName("DockerClusterApp") \
    .master("spark://spark-master:7077") \
    .getOrCreate()

def kmeans_to_spark_df(X, labels):
    """
    Convert NumPy array and cluster labels into a PySpark DataFrame with features and predictions.
    """
    # Convert NumPy data to a list of Rows with features and predictions
    rows = [Row(features=Vectors.dense(X[i]), prediction=int(labels[i])) for i in range(len(labels))]
    return spark.createDataFrame(rows)


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/12/14 14:50:35 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [8]:
spark_df = kmeans_to_spark_df(processed_data, labels)

evaluator = ClusteringEvaluator(featuresCol="features", predictionCol="prediction", metricName="silhouette")
silhouette_score = evaluator.evaluate(spark_df)

print(f"Silhouette Score: {silhouette_score}")

spark.stop()

NameError: name 'labels' is not defined

# PKMEANS

In [9]:
def assign_clusters_to_data(X, centroids):
    labels = []
    for point in X:
        distances = np.linalg.norm(centroids - point, axis=1)
        labels.append(np.argmin(distances))
    return np.array(labels)

In [10]:
import random 
import time
import math
import logging
import itertools
import typing
from numpy import array
import numpy as np
import pandas as pd
from scipy import spatial
import matplotlib.pyplot as plt
from pprint import pprint
import pyspark
from pyspark import SparkConf, SparkContext
from pyspark.sql import SparkSession
from pyspark.mllib.clustering import KMeans, KMeansModel
from itertools import groupby, compress

from typing import Tuple, Sequence

spark = SparkSession.builder \
    .appName("PKmeans") \
    .master("spark://spark-master:7077") \
    .getOrCreate()

sc = spark.sparkContext

def euclidean_dist(v1, v2):
    return np.linalg.norm(v1-v2)
    
def closest_centroid(point, centroids):
    # for dense data, we use euclidean distance
    centroid_dist_pairs = map(lambda centroid: (centroid, euclidean_dist(point, centroid)), centroids)
    return min(centroid_dist_pairs, key=lambda centroid_dist: centroid_dist[1])

def clac_mean_points_in_cluster_pandas(distances):
    # not in use. slower implementation of means calculation
    df = pd.DataFrame(distances, columns=['Point', 'Centroid', 'Distance'])
    df['Centroid'] = tuple(df['Centroid'])

    return df.groupby('Centroid')['Point'].apply(np.mean).items()

def clac_mean_points_in_cluster(distances):
    def keyfunc(item):
        return tuple(item[1])
    
    sorted_distances = sorted(distances, key=keyfunc)
    for k, g in groupby(sorted_distances, keyfunc):
        yield k, np.mean(array(list(g), dtype='object')[:,0], axis=0)

def calc_partition_centroid_means(partition, centroids):
    distances = map(lambda point: (point, 
                                   *closest_centroid(point, centroids)), 
                    partition)
    
    group = clac_mean_points_in_cluster(distances)
    return group

def closest_centroids_per_partition(data, centroids):
    """
    calculate local means for each centroid on each partition to avoid transfer of large volume of data
    
    @param data: rdd, the actual data set
    @param centroids:
    @return: rdd of tuples (original centroid, mean of points close to this centroid on each partition)
    """
    result = data.mapPartitions(lambda partition: calc_partition_centroid_means(partition, centroids.value))
    return result

def color_hash(point):
    hsh = int(point[0] ** 2 + point[1] ** 2)
    return '#' + str(hex(hsh % int('ffffff', 16)))[2:].zfill(6)

def calc_error(new_centroids_series: pd.DataFrame):
    old_and_new_centroids = array(list(new_centroids_series.items()))
    error = np.linalg.norm((old_and_new_centroids[:,0] - old_and_new_centroids[:,1]))
    return error

def aggregate_means(closest_centroids):
    df = pd.DataFrame(closest_centroids, columns=['Centroid', 'Point']) 
    new_centroids_series = df.groupby('Centroid')['Point'].apply(np.mean)
    return new_centroids_series

def pkmeans(data, n, max_iterations=15, stop_distance=0.01):
    sampled_rdd = parsed_data.sample(withReplacement=False, fraction=0.001, seed=3144)
    init_centroids = np.array(sampled_rdd.take(k))
    centroids = sc.broadcast(init_centroids)
    
    iteration = 1
    error = True
    while error > stop_distance and iteration <= max_iterations:
        loop_start = time.time()
        closest_centroids_rdd = closest_centroids_per_partition(data, centroids)
        closest_centroids = closest_centroids_rdd.collect()
        return closest_centroids

        new_centroids_series = aggregate_means(closest_centroids)
        error = calc_error(new_centroids_series)
        print("Iteration #{0}\tDistance between old and new centroids: {1:.4f}\tIteration took: {2:.4f} sec".format(
                                                                iteration, error, time.time() - loop_start))
        centroids = sc.broadcast(new_centroids_series.to_list())
        iteration += 1
    
    return new_centroids_series.to_list()
    


In [4]:
np.savetxt("data.csv", processed_data, delimiter=",")

NameError: name 'processed_data' is not defined

In [11]:
rdd = sc.textFile("data.csv")

parsed_data = rdd.map(lambda line: [float(x) for x in line.split(",")])
parsed_data = parsed_data.cache()

In [12]:
final_centroids = pkmeans(parsed_data, k)

In [ ]:
final_centroids = pkmeans(parsed_data, k)
labels = assign_clusters_to_data(processed_data, final_centroids)
spark_df = kmeans_to_spark_df(processed_data, labels)
evaluator = ClusteringEvaluator(featuresCol="features", predictionCol="prediction", metricName="silhouette")
silhouette_score = evaluator.evaluate(spark_df)

print(f"Silhouette Score: {silhouette_score}")

spark.stop()

24/12/14 14:51:29 ERROR StandaloneSchedulerBackend: Application has been killed. Reason: Master removed our application: KILLED
24/12/14 14:51:29 ERROR Inbox: Ignoring error
java.util.concurrent.RejectedExecutionException: Task java.util.concurrent.ScheduledThreadPoolExecutor$ScheduledFutureTask@398955bb[Not completed, task = java.util.concurrent.Executors$RunnableAdapter@25fc6b13[Wrapped task = org.apache.spark.scheduler.cluster.StandaloneSchedulerBackend$StandaloneDriverEndpoint$$anon$2@163228ab]] rejected from java.util.concurrent.ScheduledThreadPoolExecutor@59435f0a[Terminated, pool size = 0, active threads = 0, queued tasks = 0, completed tasks = 0]
	at java.base/java.util.concurrent.ThreadPoolExecutor$AbortPolicy.rejectedExecution(ThreadPoolExecutor.java:2065)
	at java.base/java.util.concurrent.ThreadPoolExecutor.reject(ThreadPoolExecutor.java:833)
	at java.base/java.util.concurrent.ScheduledThreadPoolExecutor.delayedExecute(ScheduledThreadPoolExecutor.java:340)
	at java.base/jav